In [114]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy.interpolate import CubicSpline
import csv
%matplotlib inline

In [115]:
#This code takes radius r as an input. It assumes that the radius positions are evenly spaced over a domain of length 1
#It uses this information to simulate heat transfer in the fin
#The outputs are the radius, temperature, volume, and heat transfer coefficient
def findState(r):
    #set up parameters
    h = 20
    k = 400
    L = 1
    r0 = 0.25
    T0 = 500
    Tinf = 200
    #process input
    N = len(r)
    x = np.linspace(0,L,N)
    dx = x[1]-x[0]
    #Solve the problem
    M = np.zeros((N,N))
    f = np.zeros((N,1))
    M[0,0] = 1
    f[0,0] = T0
    M[N-1,N-1] = 3/(2*dx)+h/k
    M[N-1,N-2] = -2/dx
    M[N-1,N-3] = 1/(2*dx)
    f[N-1,0] = Tinf*h/k
    for i in range(1,N-1):
        M[i,i+1] = 0.5*r[i]/(dx**2)+0.25*(r[i+1]-r[i-1])/(dx**2);
        M[i,i] = -r[i]/(dx**2)-h/k;
        M[i,i-1] = 0.5*r[i]/(dx**2)-0.25*(r[i+1]-r[i-1])/(dx**2);
        f[i,0] = -h*Tinf/k;
    T = np.linalg.solve(M,f)
    #Post process to find V and Q
    A = np.pi*r**2;
    Q = -k*A[0]*(-1.5*T[0]+2*T[1]-0.5*T[2])/dx;
    V = np.trapz(A,x)
    #output things
    return r,T[:,0],Q[0],V

In [116]:
#this code generates a random radius profile
#this radius can be used as a starting state for the system

#input N is the integer number of points to return the value of the radius on
#I think that 100 is a good starting value, but this is a hyperparameter that we should all try experimenting with
def makeR(N):
    n = np.random.randint(2,10)
    r_max = 0.3
    r_min = 0.03
    xp = np.linspace(0,1,n)
    rp = np.random.rand(n)*(r_max-r_min)+r_min
    x = np.linspace(0,1,N)
    rs = sp.interpolate.CubicSpline(xp,rp)
    r = rs(x)
    for i in range(0,N):
        if(r[i]>(2*r_max)):
            r[i]=2*r_max
        elif(r[i]<(0.5*r_min)):
            r[i]=0.5*r_min
    return r